In [ ]:
import csv

In [ ]:
crawls_data_filename = "data/nebula-peers-2crawls.csv"
peerids_filename = "data/all-peerids.csv"

In [ ]:
with open(peerids_filename, 'r') as file:
    nebula_peerids = {line[0]:line[1] for line in csv.reader(file)}

In [ ]:
with open(crawls_data_filename, 'r') as file:
    all_crawls = [line for line in csv.reader(file)]

In [ ]:
for p in nebula_peerids:
    print(p, nebula_peerids[p])
    break

In [ ]:
from binary_trie import Trie, bytes_to_bitstring, int_to_bitstring, bitstring_to_int, bitstring_to_bytes
import multihash as mh
import hashlib as hl
import math, os, random
import numpy as np

In [ ]:
n_buckets = 256
keylen = 32
repl = 20
concurrency = 10

In [ ]:
def peerid_to_kadid(peerid: str) -> bytes:  
    multihash = mh.from_b58_string(peerid)
    return hl.sha256(multihash).digest()

def peerid_to_bitstr(peerid: str) -> str:
    return bytes_to_bitstring(peerid_to_kadid(peerid))

def byte_to_int(b: bytes) -> int:
    return int.from_bytes(b, byteorder='big')

def bxor(b1, b2):
    n_b1 = np.frombuffer(b1, dtype='uint8')
    n_b2 = np.frombuffer(b2, dtype='uint8')

    return (n_b1 ^ n_b2).tobytes()


In [ ]:
# XOR two bitstring of equal size, the size doesn't need to be a multiple of 8
def xor_bitstring(bs0: str, bs1: str) -> str:
    s = ""
    if len(bs0) == len(bs1):
        for i in range(len(bs0)):
            if bs0[i]==bs1[i]:
                s+='0'
            else:
                s+='1'
    return s

def log_dist(bs0, bs1: bytes) -> int:
    xored = bxor(bs0, bs1)
    dist = byte_to_int(xored)
    if dist == 0:
        return n_buckets
    return n_buckets-math.floor(math.log2(dist))-1
    

Creates a dictionary for each peers peerid -> list of neighbors peerid, with peerid being the bitstring of the identity

In [ ]:
peers = {}
crawl_id = -1
for line in all_crawls:
    if crawl_id == -1:
        crawl_id = line[0]
    elif line[0] != crawl_id:
        # single crawl
        break

    peerid = peerid_to_kadid(line[2])

    buckets = {}
    for npid in line[3:]:
        neighbor = peerid_to_kadid(nebula_peerids[npid])
        bid = log_dist(peerid, neighbor)
        if bid not in buckets:
            buckets[bid] = []
        buckets[bid].append(neighbor)
    
    # only count reachable peers
    if len(buckets) > 0:
        peers[peerid] = buckets

new_peers = {}
for peer in peers:
    buckets = {}
    for bid in peers[peer]:
        existing = []
        for neighbor in peers[peer][bid]:
            if neighbor in peers:
                existing.append(neighbor)
        #peers[peer][bid] = existing
        if len(peers[peer][bid]) > 0:
            buckets[bid] = existing
    if len(buckets) > 0:
        new_peers[peer] = buckets

peers = new_peers


In [ ]:
trie = Trie()
for peer in peers:
    trie.add(bytes_to_bitstring(peer, l=n_buckets))

# Simulator

In [ ]:
def random_key():
    return os.urandom(keylen)

In [ ]:
key = random_key()

In [ ]:
closest20bs = trie.n_closest_keys(bytes_to_bitstring(key, l=n_buckets), repl)
closest20 = [bitstring_to_bytes(bs) for bs in closest20bs]

In [ ]:
closest20

## Peer and Routing Table

When looking up a key, find the 20 closest peers to that key (e.g using Binary Trie?)

Take the reference host, and check in which bucket the key falls in.

_Request_ the key to all peers from this bucket. This operation consists in checking in which bucket the target key would fall in for all _requested_ peers. Taking the union of all these buckets, we select only the 20 closest peers to key (out of 400)

The request stop when at least one of these peers is among the 20 closest computed at the begining.

In [ ]:
def n_closest(t: Trie, k: bytes, n: int):
    if t.size == 0:
        print("error nil trie")
        return
    closest = t.n_closest_keys(bytes_to_bitstring(k, l=n_buckets), n)
    return [bitstring_to_bytes(bs) for bs in closest]


In [ ]:
t = Trie()
t.add("0011")
t.add("0100")
t.add("0110")
t.add("1011")

[e for e in t.n_closest_keys("1100", 2)]

In [ ]:
# making use of peers, trie
def n_hop_lookup(peer, key):
    closest20 = n_closest(trie, key, repl)

    hop = 0
    if peer in closest20:
        return hop

    # 20 closest peers in RT
    #bid = log_dist(peer, key)
    req_trie = Trie()
    bid = log_dist(peer, key)
    count = 0

    for b in peers[peer]:
        count += len(peers[peer][b])
    #print("count", count)


    if bid in peers[peer] and len(peers[peer][bid]) > 0:
        for peer_neighbor in peers[peer][bid]:
            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
    else:
        buckets = list(peers[peer].keys())
        found_higher = False
        for b in range(bid+1, max(buckets)+1):
            if b in buckets and len(peers[peer][b]) > 0:
                for peer_neighbor in peers[peer][b]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    found_higher = True
        if not found_higher:
            print("not found higher")
            for b in range(bid-1, -1, -1):
                if b in buckets and len(peers[peer][b]) > 0:
                    for peer_neighbor in peers[peer][b]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    print("let's break")
                    break # we are only interested in the closest bucket
    req_peers = n_closest(req_trie, key, concurrency)

    while True:
        hop += 1
        #print("hop", hop)
        for c in closest20:
            if c in req_peers:
                return hop

        req_trie = Trie()
        for p in req_peers:
            bid = log_dist(p, key)
            #print(bytes_to_bitstring(p))
            #print(bytes_to_bitstring(key))
            #print("bid", bid)
            count = 0

            for b in peers[p]:
                count += len(peers[p][b])
            #print("count", count)
            #print("peer", bytes_to_bitstring(peer))
            #print("key", bytes_to_bitstring(key))

            # new attempt
            if bid in peers[p] and len(peers[p][bid]) > 0:
                for peer_neighbor in peers[p][bid]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
            else:
                buckets = list(peers[p].keys())
                found_higher = False
                for b in range(bid+1, max(buckets)+1):
                    if b in buckets and len(peers[p][b]) > 0:
                        for peer_neighbor in peers[p][b]:
                            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            found_higher = True
                if not found_higher:
                    for b in range(bid-1, -1, -1):
                        if b in buckets and len(peers[p][b]) > 0:
                            for peer_neighbor in peers[p][b]:
                                req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            break # we are only interested in the closest bucket
                    


            """
            # first check bucket, then bucket+ and finally bucket
            buckets = sorted(list(peers[p].keys()))
            i = buckets.index(bucket)
            if i == -1:
                for 
            count = 0
            max_reached = False
            while count<repl and bid>=0:
                bid = buckets[i]
                if bid in peers[p]:
                    for peer_neighbor in peers[p][bid]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                        count += 1

                if not max_reached:
                    bid += 1

            for bid in range(bucket, -1, -1):
                #print(bid)
                if bid in peers[p]:
                    for peer_neighbor in peers[p][bid]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                        count += 1
                    #print(count)
                    if count >= repl:
                        break
            
            for bid in peers[p]:
                print("candidates")
                for peer_neighbor in peers[p][bid]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    print(bytes_to_bitstring(peer_neighbor))
                print("end candidates")
            """
        old_best = req_peers[0]
        req_peers = n_closest(req_trie, key, 20)

        """
        print("old best + new candidates")
        print(bytes_to_bitstring(old_best))
        for p in req_peers:
            print(bytes_to_bitstring(p))
        print("end best")
        """


In [ ]:
# debug cell
key_bs = "1100011011110001000110010010000111110111011111111110110111001011100000111110011100001001010111001011111111000110000010100100110100101001010000110001011010100111101000000101011000011010011101000101110011000111001101100001101111000110000111100010110001000100"
peer_bs = "0011101111100111111101011000010111100110000111000011101000010000010000100001101101010000010001001001000110001101011011000011001011010010110110100101100111001011010101100001001111110000101011110111111111000111100110000100101111111000000101110001011100111101"

print("key", key_bs)

closest = trie.n_closest_keys(key_bs, repl)
print("Closest 20")
for c in closest:
    print(c)

for _ in range(1):
    res = n_hop_lookup(bitstring_to_bytes(peer_bs), bitstring_to_bytes(key_bs))
    #print(res)

In [ ]:
hop_counts = []
for i in range(10000):
    peer = random.choice(list(peers.keys()))
    key = random_key()
    n_hop = n_hop_lookup(peer, key)
    hop_counts.append(n_hop)

In [ ]:
occurences = {}
for i in hop_counts:
    if i not in occurences:
        occurences[i] = 1
    else:
        occurences[i] += 1

In [ ]:
occurences

In [ ]:
avg_current = sum(hop_counts)/len(hop_counts)
avg_current

# Balanced buckets

Note that this isn't the perfect routing table. If the observed peer has 18 reachable peers in one of its buckets, even though more peers would fit, the simulation will only have 18 balanced peers.

First try to fill the above layer, and go down if there is space left.

In [287]:
# return a list of keys
def balance_bucket(t: Trie, n: int):
    if n == 0 or t is None:
        return []
    half = math.ceil(n/2)
    if t.branch[0] is None or t.branch[1] is None:
        return [t.key]
    elif t.branch[0].size >= half and t.branch[1].size >= half:
        add0, add1 = 0, 0
        if n % 2 == 1:
            if random.randint(0, 1) == 0:
                add0 += 1
            else:
                add1 += 1
        return balance_bucket(t.branch[0], half-add0) + balance_bucket(t.branch[1], half-add1)
    elif t.branch[0].size >= half:
        b1 = t.branch[1].size
        return balance_bucket(t.branch[0], n-b1) + balance_bucket(t.branch[1], b1)
    elif t.branch[1].size >= half:
        b0 = t.branch[0].size
        return balance_bucket(t.branch[0], b0) + balance_bucket(t.branch[1], n-b0)
    else:
        return t.match_prefix_keys(prefix="")

In [ ]:
trie_test = Trie()
nodeIDs = [2, 3, 4, 6, 7, 9, 11, 13]
for i in nodeIDs:
    trie_test.add(int_to_bitstring(i, 4))

print(balance_bucket(trie_test, 4))

In [ ]:
balanced_peers = {}
for peer in peers:
    
    buckets = {}
    for bid in peers[peer]:
        n = len(peers[peer][bid])
        prefix = bytes_to_bitstring(peer)[:bid]
        t = trie.find_trie(key=prefix)
        buckets[bid] = [bitstring_to_bytes(k) for k in balance_bucket(t, n)]

    balanced_peers[peer] = buckets


In [ ]:
len(peers)==len(balanced_peers)

In [ ]:
# making use of peers, trie
def n_hop_lookup_with_peers(peer, key, peers):
    closest20 = n_closest(trie, key, repl)

    hop = 0
    if peer in closest20:
        return hop

    req_trie = Trie()
    bid = log_dist(peer, key)

    if bid in peers[peer] and len(peers[peer][bid]) > 0:
        for peer_neighbor in peers[peer][bid]:
            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
    else:
        buckets = list(peers[peer].keys())
        found_higher = False
        for b in range(bid+1, max(buckets)+1):
            if b in buckets and len(peers[peer][b]) > 0:
                for peer_neighbor in peers[peer][b]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    found_higher = True
        if not found_higher:
            for b in range(bid-1, -1, -1):
                if b in buckets and len(peers[peer][b]) > 0:
                    for peer_neighbor in peers[peer][b]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    break # we are only interested in the closest bucket
    req_peers = n_closest(req_trie, key, concurrency)

    while True:
        hop += 1
        for c in closest20:
            if c in req_peers:
                return hop

        req_trie = Trie()
        for p in req_peers:
            bid = log_dist(p, key)

            if bid in peers[p] and len(peers[p][bid]) > 0:
                for peer_neighbor in peers[p][bid]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
            else:
                buckets = list(peers[p].keys())
                found_higher = False
                for b in range(bid+1, max(buckets)+1):
                    if b in buckets and len(peers[p][b]) > 0:
                        for peer_neighbor in peers[p][b]:
                            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            found_higher = True
                if not found_higher:
                    for b in range(bid-1, -1, -1):
                        if b in buckets and len(peers[p][b]) > 0:
                            for peer_neighbor in peers[p][b]:
                                req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            break # we are only interested in the closest bucket
                    
        req_peers = n_closest(req_trie, key, 20)

In [ ]:
hop_counts_balanced = []
for i in range(10000):
    peer = random.choice(list(peers.keys()))
    key = random_key()
    n_hop = n_hop_lookup_with_peers(peer, key, balanced_peers)
    hop_counts_balanced.append(n_hop)

In [ ]:
occurences = {}
for i in hop_counts_balanced:
    if i not in occurences:
        occurences[i] = 1
    else:
        occurences[i] += 1

In [ ]:
occurences

In [ ]:
avg_balanced = sum(hop_counts_balanced)/len(hop_counts_balanced)
avg_balanced

In [ ]:
print("Average current buckets", avg_current)
print("Average balanced buckets", avg_balanced)
print(1-avg_current/avg_balanced)

## Results

Balanced buckets appear to have on average 5% more hops than unbalanced ones. Maybe nodes that are in many other's RT have a well furnished RT themselves. However, when nodes are selected randomly to 

# Random setting simulation

The following experiment doesn't take as input the state of the network gathered by Nebula. It is solely a theoretical simulation.

In [ ]:
peers_number = 25000
sim_concurrency = 10
sim_repl = 20
bucket_size = 20

# Balanced buckets don't seem to work (at least for high id buckets)

In [291]:
sim_trie = Trie()
sim_peers = []

sim_peers_random = {}
sim_peers_balanced = {}

for _ in range(peers_number):
    peerid = random_key()
    sim_peers.append(peerid)
    sim_trie.add(bytes_to_bitstring(peerid))

    sim_peers_random[peerid] = {}
    sim_peers_balanced[peerid] = {}


for bid in range(20):
    for i in range(2**(bid+1)):
        candidate_prefix = int_to_bitstring(i, bid+1)
        candidates = sim_trie.match_prefix_keys(candidate_prefix)

        # flip last bit
        target_prefix = candidate_prefix[:-1]
        if candidate_prefix[-1] == "0":
            target_prefix += "1"
        else:
            target_prefix += "0"

        targets = sim_trie.match_prefix_keys(target_prefix)

        subtrie_root = sim_trie.find_trie(candidate_prefix)

        for t in targets:
            # set random buckets
            random_bucket = [bitstring_to_bytes(p) for p in random.sample(candidates, min(bucket_size, len(candidates)))]
            sim_peers_random[bitstring_to_bytes(t)][bid] = random_bucket
            # set balanced buckets
            if subtrie_root is None:
                balanced_bucket = [bitstring_to_bytes(p) for p in balance_bucket(subtrie_root, bucket_size)]
                sim_peers_balanced[bitstring_to_bytes(t)][bid] = balanced_bucket

        


In [ ]:
sim_peers_random

In [313]:
# making use of peers, trie
def n_hop_lookup_with_peers_and_trie(peer, key, peers, trie):
    closest20 = n_closest(trie, key, repl)

    hop = 0
    if peer in closest20:
        return hop

    req_trie = Trie()
    bid = log_dist(peer, key)

    if bid in peers[peer] and len(peers[peer][bid]) > 0:
        for peer_neighbor in peers[peer][bid]:
            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
    else:
        buckets = list(peers[peer].keys())
        found_higher = False
        for b in range(bid+1, max(buckets)+1):
            if b in buckets and len(peers[peer][b]) > 0:
                for peer_neighbor in peers[peer][b]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    found_higher = True
        if not found_higher:
            for b in range(bid-1, -1, -1):
                if b in buckets and len(peers[peer][b]) > 0:
                    for peer_neighbor in peers[peer][b]:
                        req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                    break # we are only interested in the closest bucket
    if req_trie.size == 0:
        print(peers[peer])
        print("empty trie")
    req_peers = n_closest(req_trie, key, concurrency)

    while True:
        hop += 1
        for c in closest20:
            if c in req_peers:
                return hop

        req_trie = Trie()
        for p in req_peers:
            bid = log_dist(p, key)

            if bid in peers[p] and len(peers[p][bid]) > 0:
                for peer_neighbor in peers[p][bid]:
                    req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
            else:
                buckets = list(peers[p].keys())
                found_higher = False
                for b in range(bid+1, max(buckets)+1):
                    if b in buckets and len(peers[p][b]) > 0:
                        for peer_neighbor in peers[p][b]:
                            req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            found_higher = True
                if not found_higher:
                    for b in range(bid-1, -1, -1):
                        if b in buckets and len(peers[p][b]) > 0:
                            for peer_neighbor in peers[p][b]:
                                req_trie.add(bytes_to_bitstring(peer_neighbor, l=n_buckets))
                            break # we are only interested in the closest bucket
                    
        req_peers = n_closest(req_trie, key, 20)

In [301]:
sim_random_hop_count = []
for i in range(10000):
    peer = random.choice(sim_peers)
    key = random_key()
    n_hop = n_hop_lookup_with_peers_and_trie(peer, key, sim_peers_random, sim_trie)
    sim_random_hop_count.append(n_hop)


In [302]:
sim_avg_random = sum(sim_random_hop_count)/len(sim_random_hop_count)
sim_avg_random

1.9352

In [314]:
sim_balanced_hop_count = []
for i in range(10000):
    peer = random.choice(sim_peers)
    key = random_key()
    n_hop = n_hop_lookup_with_peers_and_trie(peer, key, sim_peers_balanced, sim_trie)
    sim_balanced_hop_count.append(n_hop)

{12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: []}
empty trie
error nil trie


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
sim_avg_random = sum(sim_random_hop_count)/len(sim_random_hop_count)
sim_avg_random